Notes:

> This model is not that good, needs improvement.

> Without class *Industrial*, model fared much better than current state.

> Will add a few more CNN layers and decrease the Dropout rates.

> Version 1-2: Trained using user-defined model

> Version 4: Testing using ResNet101

> Version 5: Training with ResNet-50

> Version 7-8: Training using the EuroSat-ResNet50 Model imported from TensorFlow Hub

Issues:

> Misclassification with ResNet101 is high with original layers trainability set as false.

> Need to load ImageNet weights

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import os, cv2, json, random, itertools

from tqdm import tqdm
from IPython.display import SVG
from tensorflow.keras.utils import plot_model, model_to_dot
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical, Sequence
from sklearn.metrics import f1_score

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Add, Input, Conv2D, Dropout, Activation, BatchNormalization, MaxPooling2D, ZeroPadding2D, AveragePooling2D, Flatten, Dense)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, Callback, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import *
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
def show_final_history(history):
    
    plt.style.use("ggplot")
    fig, ax = plt.subplots(1,2,figsize=(15,5))
    
    ax[0].set_title('Loss')
    ax[1].set_title('Accuracy')
    
    ax[0].plot(history.history['loss'], "r-", label='Training Loss')
    ax[0].plot(history.history['val_loss'], "g-", label='Validation Loss')
    ax[1].plot(history.history['accuracy'], "r-", label='Training Accuracy')
    ax[1].plot(history.history['val_accuracy'], "g-", label='Validation Accuracy')
    
    ax[0].legend(loc='upper right')
    ax[1].legend(loc='lower right')
    
    plt.show();
    pass

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar();
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f'
    thresh = cm.max()/2.0
    
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        
        plt.text(j,i, format(cm[i,j], fmt),
                horizontalalignment = "center",
                color = "white" if cm[i,j] > thresh else "black")
        pass
    
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.grid(False);
    pass

In [ ]:
base_path = "../input/eurosat-dataset/EuroSAT"

def data_generator(csv_file, num_classes, batch_size = 10, target_size=64):
    
    df = pd.read_csv(csv_file)
    df.drop(columns=df.columns[0], inplace=True)
    num_samples = df.shape[0]
    
    while True:
        
        for offset in range(0, num_samples, batch_size):
            batch_samples_idx = df.index[offset:offset+batch_size]
            
            X = []
            y = []
            
            for i in batch_samples_idx:
                img_name = df.loc[i,'Filename']
                img = load_img(os.path.join(base_path,img_name),target_size=(224,224))
                img = img_to_array(img)
                
                img = preprocess_input(img)
                label = df.loc[i,'Label']
                
                X.append(img)
                y.append(label)
                pass
            
            X = np.array(X)
            y = np.array(y)
            y = to_categorical(y, num_classes=num_classes)
            
            yield X, y
            pass
        pass
    pass

In [ ]:
train_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSAT/train.csv", num_classes=10, batch_size=10)
val_generator = data_generator(csv_file="../input/eurosat-dataset/EuroSAT/validation.csv", num_classes=10, batch_size=10)

In [ ]:
with open("../input/eurosat-dataset/EuroSAT/label_map.json","r") as f:
    class_names_encoded = json.load(f)
    pass

class_names = list(class_names_encoded.keys())
num_classes = len(class_names)
class_names_encoded

In [ ]:
model = Sequential([
    Input((224,224,3)),
    hub.KerasLayer("https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1"),
    Dropout(0.2),
    Dense(len(class_names), activation="softmax")
])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint("eurosat_resnet50_model_weights.h5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
logs = TensorBoard("eurosat_resnet-logs")

In [ ]:
train_df = pd.read_csv("../input/eurosat-dataset/EuroSAT/train.csv")
train_labels = train_df.loc[:,'Label']
train_labels = np.array(train_labels)

num_train_samples = train_labels.shape[0]

val_df = pd.read_csv("../input/eurosat-dataset/EuroSAT/validation.csv")
val_labels = val_df.loc[:,'Label']
val_labels = np.array(val_labels)

num_val_samples = val_labels.shape[0]

num_train_samples, num_val_samples

In [ ]:
np.unique(train_labels, return_counts=True)

In [ ]:
train_labels_encoded = to_categorical(train_labels,num_classes=10)

classTotals = train_labels_encoded.sum(axis=0)
classWeight = {}

for i in range(len(classTotals)):
    classWeight[i] = classTotals[i]/classTotals.max()
    pass

classWeight

In [ ]:
class CyclicLR(Callback):
    
    def __init__(self, base_lr=0.0001,max_lr=0.001,step_size=2000.,mode="triangular",
              gamma=1.,scale_fn=None, scale_mode='cycle'):
        
        super(CyclicLR, self).__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x:1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history_lr = {}
        
        self._reset()
        pass
    
    
    def _reset(self, new_base_lr=None,new_max_lr=None,new_step_size=None):
        
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        
        self.clr_iterations = 0.
        pass
    
    
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        pass
    
    def on_train_begin(self, logs={}):
        
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
        pass
    
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        
        self.history_lr.setdefault('lr',[]).append(K.get_value(self.model.optimizer.lr))
        self.history_lr.setdefault('iterations', []).append(self.trn_iterations)
        
        for (k,v) in logs.items():
            self.history_lr.setdefault(k, []).append(v)
            pass
        
        K.set_value(self.model.optimizer.lr, self.clr())
        pass

In [ ]:
# class StepDecay(LearningRateScheduler):
#     def __init__(self, initAlpha=0.001, factor=0.1, dropEvery=30):
#         self.initAlpha = initAlpha
#         self.factor = factor
#         self.dropEvery = dropEvery
#         pass
    
#     def __cal__(self, epoch):
#         exp = np.floor((1+epoch)/self.dropEvery)
#         alpha = self.initAlpha * (self.factor ** exp)
        
#         return float(alpha)

In [ ]:
clr = CyclicLR(base_lr=0.0001,max_lr=0.001,step_size=2000.,mode='triangular')

In [ ]:
opt = Adam(lr=1e-5)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs = 1000
batchSize = 10

history = model.fit(train_generator,
                   steps_per_epoch= num_train_samples//batchSize,
                   epochs = epochs,
                   verbose = 1,
                   validation_data = val_generator,
                   validation_steps = num_val_samples//batchSize,
                   callbacks = [checkpoint, logs],
                   class_weight=classWeight
                   )

In [ ]:
show_final_history(history)

In [ ]:
def obtain_images(csv_file):
    
    df = pd.read_csv(csv_file)
    df.drop(columns=df.columns[0], inplace=True)
    num_samples = df.shape[0]
        
    X = []
    y = []

    for i in tqdm(range(num_samples)):
        img_name = df.loc[i,'Filename']
        img = cv2.imread(os.path.join(base_path,img_name))
        img = cv2.resize(img,(64,64))
        label = df.loc[i,'Label']
        
        X.append(img)
        y.append(label)
        pass
    
    X = np.array(X)
    y = np.array(y)
    
    return X,y
    pass

In [ ]:
test_images, test_labels = obtain_images(csv_file="../input/eurosat-dataset/EuroSAT/test.csv")

test_images.shape, test_labels.shape

In [ ]:
test_encoded = to_categorical(test_labels,num_classes = 10)
test_encoded.shape

In [ ]:
test_pred = model.predict(test_images)
test_pred = np.argmax(test_pred, axis=1)
test_pred.shape

In [ ]:
val_evaluate = model.evaluate(test_images, test_encoded)

print("Loss: {}, Accuracy: {}".format(val_evaluate[0], val_evaluate[1]))

In [ ]:
cnf_mat = confusion_matrix(test_labels, test_pred)

plt.figure()
plot_confusion_matrix(cnf_mat, classes=class_names)
plt.grid(False);
plt.show();

In [ ]:
for f1,class_name in zip(f1_score(test_labels, test_pred, average=None), class_names):
    print("Class name: {}, F1 score: {:.3f}".format(class_name, f1))
    pass

In [ ]:
model.save("eurosat_resnet_transfer_model_v5.h5")